# Getting Started

## Import Packages

In [19]:
import sys, os
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
import json
import plotly.graph_objects as go
import plotly.express as px
import qgrid
from plotly.offline import iplot
from scipy import stats
from ipyleaflet import (Map, GeoData, basemaps, WidgetControl, GeoJSON,
 LayersControl, Icon, Marker,basemap_to_tiles, Choropleth,
 MarkerCluster, Heatmap,SearchControl, 
 FullScreenControl)
from ipywidgets import Text, HTML, widgets, interactive, HBox, VBox
from branca.colormap import linear
%matplotlib inline

## Get Data

Load metric analysis data

In [20]:
CD = pd.read_csv("All_DAs_updated_metrics_20201023.csv")
MA = pd.read_csv("MNA_Metrics_10.19.2020.csv")

Merge data into a single dataframe called `DA_Data`

In [21]:
MA.set_index('STORMNET_I', inplace=True)
CD.set_index('STORMNET_ID', inplace=True)
DA_Data = pd.concat([MA, CD], axis=1, sort=False).reset_index()

Cleanup fields

In [22]:
DA_Data = DA_Data.drop(['OBJECTID_1','Type_AS', 'Type_BR', 'Type_DP', 'Type_FB', 'Type_FM', 'Type_GR', 'Type_MB', 'Type_OS', 'Type_Other', 'Type_PL', 'Type_PP', 'Type_RF', 'Type_RT', 'Type_SF', 'Type_TF', 'Type_TR', 'Type_UG', 'Type_VF', 'Type_VS', 'Type_WL', 'Type_WS', 'Type_WP', 'Shape_Length', 'Shape_Area', 'Sum_DRAINAGE_AREA'], axis=1)
DA_Data = DA_Data.rename(columns={"index":"STORMNET_ID","Count_PerDA":"BMP_Per_DA","Sum_TREATED_AREA":"Treated_Acres","Pct_Treated":"Percent_Treated","Area_AC":"Drainage_Area_AC","total_acres":"Area_AC","vdot_bmp_count":"VDOT_BMP_Count","swe_perc":"Percent_STW_Easement","row_perc":"Percent_ROW","total_convey_length_ft":"Conveyance_Len_FT","swe_convey_length_ft":"STW_Conveyance_Len_FT","total_county_acres":"County_Acres","imperv_county_acres":"Impervious_Acres"})
DA_Data = DA_Data.loc[:,~DA_Data.columns.duplicated()]

View Data

In [23]:
DA_Data

,STORMNET_ID,Owner,Impervious_Acres


Summary statistics for all drainage areas

In [24]:
DA_Data.loc[:, DA_Data.columns != 'STORMNET_ID'].describe().round(2)

,BMP_Per_DA,Treated_Acres,Percent_Treated,Drainage_Area_AC,Area_AC,VDOT_BMP_Count,Percent_STW_Easement,Percent_ROW,Conveyance_Len_FT,STW_Conveyance_Len_FT,row_convey_length_ft,County_Acres,Impervious_Acres


Categorize drainage area ownership into 5 categories:

- FFX
- VDOT
- Private
- Other Jurisdiction
- VDOT/FFX

In [25]:
my_dict = {'VDOT':'VDOT','Private':'Private','Loudoun County':'Other Jurisdiction','Unknown':'Private','FFX':'FFX','Other Jurisdiction':'Other Jurisdiction', 'VDOt/FFX':'VDOT/FFX','Town of Vienna':'Other Jurisdiction','WMATA':'Other Jurisdiction','VPDES':'Other Jurisdiction','Arlington County':'Other Jurisdiction', 'City of Alexandria':'Other Jurisdiction','FFX Board of Supervisors':'FFX'}
DA_Data = DA_Data.replace({"Owner": my_dict})

View summary statistics by 'Maint'

In [26]:
Spreadsheet_df = DA_Data.groupby('Owner').describe().unstack(1).reset_index()
Spreadsheet_df = Spreadsheet_df.rename(columns={"level_0":"Metric","level_1":"Statistic","Owner":"Maint","0":"Value"})
qgrid_widget = qgrid.show_grid(Spreadsheet_df, show_toolbar=True)
qgrid_widget

AttributeError: module 'pandas.core' has no attribute 'index'

In [ ]:
g=sns.displot(DA_Data, x="Percent_ROW", hue="Owner", element="step")
g = (g.set_axis_labels("Percent ROW","Drainage Area Count"))
g.fig.set_size_inches(15,9)

In [ ]:
g=sns.displot(DA_Data, x="Percent_ROW", hue="Owner", element="step")
g = (g.set_axis_labels("Percent ROW","Drainage Area Count").set(xlim=(2,100),ylim=(0,500)))
g.fig.set_size_inches(15,9)

In [ ]:
Maint = widgets.Dropdown(
    options=list(DA_Data['Owner'].unique()),
    value='FFX',
    description='Owner:',
)

# Assign an empty figure widget with two traces
trace1 = go.Histogram(x=DA_Data['Percent_ROW'], opacity=0.75, name='Percent_ROW')
trace2 = go.Histogram(x=DA_Data['Percent_STW_Easement'], opacity=0.75, name='Percent_STW_Easement')
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text='ROW and Easement Data'
                        ),
                        barmode='overlay'
                    ))

In [ ]:
def validate():
    if Maint.value in DA_Data['Owner'].unique():
        return True
    else:
        return False

In [ ]:
def response(change):
    if validate():
        if Maint.value == 'VDOT':
            temp_df = DA_Data.loc[DA_Data['Owner'] == 'VDOT', ['Percent_ROW', 'Percent_STW_Easement']]
        elif Maint.value == 'Private':
            temp_df = DA_Data.loc[DA_Data['Owner'] == 'Private', ['Percent_ROW', 'Percent_STW_Easement']]
        elif Maint.value == 'Other Jurisdiction':
            temp_df = DA_Data.loc[DA_Data['Owner'] == 'Other Jurisdiction', ['Percent_ROW', 'Percent_STW_Easement']]
        elif Maint.value == 'FFX':
            temp_df = DA_Data.loc[DA_Data['Owner'] == 'FFX', ['Percent_ROW', 'Percent_STW_Easement']]
        else:
            temp_df = DA_Data.loc[DA_Data['Owner'] == 'VDOT/FFX', ['Percent_ROW', 'Percent_STW_Easement']]
        x1 = temp_df['Percent_ROW']
        x2 = temp_df['Percent_STW_Easement']
        with g.batch_update():
            g.data[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Percent'
            g.layout.yaxis.title = 'Number of Drainage Areas'

Maint.observe(response, names="value")

In [ ]:
widgets.VBox([Maint, g])

In [ ]:
# Map new column to dataframe to add colors by owner
col_dict = {'VDOT':'rgb(255,128,0)', 'VDOT/FFX':'rgb(255,0,255)', 'Private':'rgb(0,0,255)', 'Other Jurisdiction':'rgb(32,32,32)', 'FFX':'rgb(0,204,0)'}
cols = DA_Data['Owner'].map(col_dict)

f = go.FigureWidget(go.Scatter(y = DA_Data['Percent_ROW'],
                                x = DA_Data['Percent_ROW'],
                                mode = 'markers',
                               marker=dict(size=8, color=cols),
                              text=DA_Data['Owner']))
scatter = f.data[0]
N = len(DA_Data)
scatter.x = scatter.x + np.random.rand(N)/10 *(DA_Data['Percent_ROW'].max() - DA_Data['Percent_ROW'].min())
scatter.y = scatter.y + np.random.rand(N)/10 *(DA_Data['Percent_ROW'].max() - DA_Data['Percent_ROW'].min())
scatter.marker.opacity = 0.8

def update_axes(xaxis, yaxis):
    scatter = f.data[0]
    scatter.x = DA_Data[xaxis]
    scatter.y = DA_Data[yaxis]
    with f.batch_update():
        f.layout.xaxis.title = xaxis
        f.layout.yaxis.title = yaxis
        scatter.x = scatter.x + np.random.rand(N)/10 *(DA_Data[xaxis].max() - DA_Data[xaxis].min())
        scatter.y = scatter.y + np.random.rand(N)/10 *(DA_Data[yaxis].max() - DA_Data[yaxis].min())

axis_dropdowns = interactive(update_axes, yaxis = DA_Data.select_dtypes('int64').columns | DA_Data.select_dtypes('float64').columns, xaxis = DA_Data.select_dtypes('int64').columns | DA_Data.select_dtypes('float64').columns)


# Put everything together
VBox((HBox(axis_dropdowns.children),f))


In [ ]:
VDOT_Data = DA_Data.loc[DA_Data['Owner'] == 'VDOT']
VDOT_FFX_Data = DA_Data.loc[DA_Data['Owner'] == 'VDOT/FFX']
Private_Data = DA_Data.loc[DA_Data['Owner'] == 'Private']
Other_Jurisdiction_Data = DA_Data.loc[DA_Data['Owner'] == 'Other Jurisdiction']
FFX_Data = DA_Data.loc[DA_Data['Owner'] == 'FFX']

In [ ]:
# Histogram Metrics
my_list = list(DA_Data.columns.values.tolist())
newlist = my_list[2:]


Metric = widgets.Dropdown(
    options=newlist,
    value='Treated_Acres',
    description='Metric:',
)

# Assign an empty figure widget with five traces
trace1 = go.Histogram(x=VDOT_Data['Treated_Acres'], opacity=0.75, name='VDOT')
trace2 = go.Histogram(x=VDOT_FFX_Data['Treated_Acres'], opacity=0.75, name='VDOT/FFX')
trace3 = go.Histogram(x=Private_Data['Treated_Acres'], opacity=0.75, name='Private')
trace4 = go.Histogram(x=Other_Jurisdiction_Data['Treated_Acres'], opacity=0.75, name='Other Jurisdiction')
trace5 = go.Histogram(x=FFX_Data['Treated_Acres'], opacity=0.75, name='FFX')
h = go.FigureWidget(data=[trace1,trace2,trace3,trace4,trace5],
                    layout=go.Layout(
                        title=dict(
                            text='Metric Histogram'
                        ),
                        barmode='overlay'
                    ))
def response(change):
    if Metric.value == 'Treated_Acres':
        VDOT_df = VDOT_Data['Treated_Acres']
        VDOT_FFX_df = VDOT_FFX_Data['Treated_Acres']
        Private_df = Private_Data['Treated_Acres']
        Other_Jurisdiction_df = Other_Jurisdiction_Data['Treated_Acres']
        FFX_df = FFX_Data['Treated_Acres']
    elif Metric.value == 'Percent_Treated':
        VDOT_df = VDOT_Data['Percent_Treated']
        VDOT_FFX_df = VDOT_FFX_Data['Percent_Treated']
        Private_df = Private_Data['Percent_Treated']
        Other_Jurisdiction_df = Other_Jurisdiction_Data['Percent_Treated']
        FFX_df = FFX_Data['Percent_Treated']
    elif Metric.value == 'Drainage_Area_AC':
        VDOT_df = VDOT_Data['Drainage_Area_AC']
        VDOT_FFX_df = VDOT_FFX_Data['Drainage_Area_AC']
        Private_df = Private_Data['Drainage_Area_AC']
        Other_Jurisdiction_df = Other_Jurisdiction_Data['Drainage_Area_AC']
        FFX_df = FFX_Data['Drainage_Area_AC']
    elif Metric.value == 'Area_AC':
        VDOT_df = VDOT_Data['Area_AC']
        VDOT_FFX_df = VDOT_FFX_Data['Area_AC']
        Private_df = Private_Data['Area_AC']
        Other_Jurisdiction_df = Other_Jurisdiction_Data['Area_AC']
        FFX_df = FFX_Data['Area_AC']
    elif Metric.value == 'VDOT_BMP_Count':
        VDOT_df = VDOT_Data['VDOT_BMP_Count']
        VDOT_FFX_df = VDOT_FFX_Data['VDOT_BMP_Count']
        Private_df = Private_Data['VDOT_BMP_Count']
        Other_Jurisdiction_df = Other_Jurisdiction_Data['VDOT_BMP_Count']
        FFX_df = FFX_Data['VDOT_BMP_Count']    
    elif Metric.value == 'Percent_STW_Easement':
        VDOT_df = VDOT_Data['Percent_STW_Easement']
        VDOT_FFX_df = VDOT_FFX_Data['Percent_STW_Easement']
        Private_df = Private_Data['Percent_STW_Easement']
        Other_Jurisdiction_df = Other_Jurisdiction_Data['Percent_STW_Easement']
        FFX_df = FFX_Data['Percent_STW_Easement']      
    elif Metric.value == 'Percent_ROW':
        VDOT_df = VDOT_Data['Percent_ROW']
        VDOT_FFX_df = VDOT_FFX_Data['Percent_ROW']
        Private_df = Private_Data['Percent_ROW']
        Other_Jurisdiction_df = Other_Jurisdiction_Data['Percent_ROW']
        FFX_df = FFX_Data['Percent_ROW']   
    elif Metric.value == 'STW_Conveyance_Len_FT':
        VDOT_df = VDOT_Data['STW_Conveyance_Len_FT']
        VDOT_FFX_df = VDOT_FFX_Data['STW_Conveyance_Len_FT']
        Private_df = Private_Data['STW_Conveyance_Len_FT']
        Other_Jurisdiction_df = Other_Jurisdiction_Data['STW_Conveyance_Len_FT']
        FFX_df = FFX_Data['STW_Conveyance_Len_FT']       
    elif Metric.value == 'row_convey_length_ft':
        VDOT_df = VDOT_Data['row_convey_length_ft']
        VDOT_FFX_df = VDOT_FFX_Data['row_convey_length_ft']
        Private_df = Private_Data['row_convey_length_ft']
        Other_Jurisdiction_df = Other_Jurisdiction_Data['row_convey_length_ft']
        FFX_df = FFX_Data['row_convey_length_ft']          
    elif Metric.value == 'County_Acres':
        VDOT_df = VDOT_Data['County_Acres']
        VDOT_FFX_df = VDOT_FFX_Data['County_Acres']
        Private_df = Private_Data['County_Acres']
        Other_Jurisdiction_df = Other_Jurisdiction_Data['County_Acres']
        FFX_df = FFX_Data['County_Acres']      
    else:
        VDOT_df = VDOT_Data['Impervious_Acres']
        VDOT_FFX_df = VDOT_FFX_Data['Impervious_Acres']
        Private_df = Private_Data['Impervious_Acres']
        Other_Jurisdiction_df = Other_Jurisdiction_Data['Impervious_Acres']
        FFX_df = FFX_Data['Impervious_Acres']   
    x1 = VDOT_df
    x2 = VDOT_FFX_df
    x3 = Private_df
    x4 = Other_Jurisdiction_df
    x5 = FFX_df
    with h.batch_update():
        h.data[0].x = x1
        h.data[1].x = x2
        h.data[2].x = x3
        h.data[3].x = x4
        h.data[4].x = x5
        h.layout.barmode = 'overlay'
        h.layout.xaxis.title = 'Metric'
        h.layout.yaxis.title = 'Number of Drainage Areas'

Metric.observe(response, names="value")

In [ ]:
widgets.VBox([Metric, h])